# Make MOHID bathymetry file

In [1]:
import numpy as np
import xarray as xr

In [2]:
grid = xr.open_dataset('https://salishsea.eos.ubc.ca/erddap/griddap/ubcSSn2DMeshMaskV17-02')
bathymetry = xr.open_dataset('AfterNEMOBathy201702.nc').Bathymetry.values[1:897, 1:397]

In [3]:
condlist = [bathymetry == 0, bathymetry != 0]
choicelist = [-99, bathymetry]
bathymetry = np.select(condlist, choicelist)

In [4]:
left = bathymetry[...,0]
condlist = [left == -99, left != -99]
choicelist = [-99, 0]
bathymetry[...,0] = np.select(condlist, choicelist)

right = bathymetry[...,-1]
condlist = [right == -99, right != -99]
choicelist = [-99, 0]
bathymetry[...,-1] = np.select(condlist, choicelist)

top = bathymetry[0]
condlist = [top == -99, top != -99]
choicelist = [-99, 0]
bathymetry[0] = np.select(condlist, choicelist)

bottom = bathymetry[-1]
condlist = [bottom == -99, bottom != -99]
choicelist = [-99, 0]
bathymetry[-1] = np.select(condlist, choicelist)

In [5]:
u = grid.gphiu.isel(time = 0).values
u_roll = np.roll(u,shift = -1, axis = 0)
unstaggered_u = ((u + u_roll)/2)[0:897, 1:398]

In [6]:
v = grid.glamv.isel(time = 0).values 
v_roll = np.roll(v,shift = -1, axis = 1)
unstaggered_v = ((v + v_roll)/2)[0:897, 0:397]

## confirm

In [7]:
a = np.array([[1,2,3],
              [4,5,6],
              [7,8,9]])
a_roll = np.roll(a,shift = -1, axis = 0)
unstagerred_a = ((a + a_roll)/2)[0:2, 1:3]

In [8]:
assert((unstagerred_a == np.array([[3.5, 4.5],[6.5, 7.5]])).all())

In [9]:
b = np.array([[1,2,3],
              [4,5,6],
              [7,8,9]])
b_roll = np.roll(b,shift = -1, axis = 1)
unstagerred_b = ((b + b_roll)/2)[0:2, 0:2]

In [10]:
assert((unstagerred_b == np.array([[1.5, 2.5],[4.5, 5.5]])).all())

## write to file

In [135]:
bathy = open('Bathymetry.dat', 'wt')

bathy.write('ILB_IUB : 1 896\n')
bathy.write('JLB_JUB : 1 396\n')
bathy.write('COORD_TIP : 4\n')
bathy.write(f'ORIGIN : {unstaggered_v[0][0]} {unstaggered_u[0][0]}\n')
bathy.write('GRID_ANGLE : 0\n')
bathy.write('LATITUDE : 0\n')
bathy.write('LONGITUDE: 0\n')
bathy.write('FILL_VALUE : -99\n')
bathy.write('<CornersXY>\n')
for i in range(897):
    for j in range(397):
        bathy.write(f'{unstaggered_v[i][j]} {unstaggered_u[i][j]}\n')
bathy.write('<BeginGridData2D>\n')
for i in range(896):
    for j in range(396):
        bathy.write(f'{bathymetry[i][j]}\n')
bathy.write('<EndGridData2D>\n')
bathy.close()